# Test
This notebook contains some tests to ensure mathGAP is working properly

In [1]:
from typing import List

from mathgap.logicalforms import Container, Transfer, Comp, PartWhole, CompEq
from mathgap.trees.sampling import CanonicalOrderSampler, VariableTimeBasedSampler
from mathgap.properties import PropertyType, PropertyKey
from mathgap.natlang.templates import TextPart, render_problem_and_answers, Origin, PropertyKeysOrigin, render_reasoning_trace, TemplateType
from mathgap.generation_util import default_generator, default_instantiator, default_templates_and_samplers

In [2]:
seed = 14
dataversion="v1"

## Inspect Templates
Check all templates for correctness

In [3]:
instantiator = default_instantiator(dataversion=dataversion)
ps_template_sampler, ps_answers_template_sampler, ps_renderer, rt_template_sampler, rt_renderer = default_templates_and_samplers(dataversion=dataversion)

In [4]:
catalog = ps_template_sampler.sampler.template_catalog
for lf in catalog.templates_by_lf_and_type:
    print(lf.__name__)
    print("\tStatements:")
    for t in catalog.get_templates_by_lf_and_type(lf, template_type=TemplateType.STATEMENT):
        print(f"\t- {''.join([str(p) for p in t.parts])}\n\t  iff {t.condition}")

    print()
    print("\tConclusions:")
    for t in catalog.get_templates_by_lf_and_type(lf, template_type=TemplateType.CONCLUSION):
        print(f"\t- {''.join([str(p) for p in t.parts])}\n\t  iff {t.condition}")

    print()
    print("\tQuestions:")
    for t in catalog.get_templates_by_lf_and_type(lf, template_type=TemplateType.QUESTION):
        print(f"\t- {''.join([str(p) for p in t.parts])}\n\t  iff {t.condition}")

Container
	Statements:
	- [agent] has [quantity] [entity].
	  iff True
	- [agent] has [quantity] [attribute] [entity].
	  iff True
	- [agent] has [quantity] [unit]s of [entity].
	  iff True

	Conclusions:
	- Therefore, [agent] now has [quantity;depth=1] = [quantity] [entity].
	  iff True
	- Therefore, [agent] now has [quantity;depth=1] = [quantity] [attribute] [entity].
	  iff True
	- Therefore, [agent] now has [quantity;depth=1] = [quantity] [unit]s of [entity].
	  iff True

	Questions:
	- How many [entity] does [agent] have?
	  iff True
	- How many [attribute] [entity] does [agent] have?
	  iff True
	- How many [unit]s of [entity] does [agent] have?
	  iff True
Transfer
	Statements:
	- Then, [receiver] gets another [quantity] [entity] from [sender].
	  iff ([receiver] == [*conclusion.agent])
	- Then, [receiver] gets another [quantity] [attribute] [entity] from [sender].
	  iff ([receiver] == [*conclusion.agent])
	- Then, [receiver] gets another [quantity] [unit]s of [entity] from [se

## Inspect Reordering

In [5]:
def visualize_character_origins(origins: List[Origin]) -> str:
    p_type_rep = {
        PropertyType.AGENT: "a",
        PropertyType.QUANTITY: "n",
        PropertyType.ENTITY: "e",
        PropertyType.ATTRIBUTE: "@",
        PropertyType.UNIT: "u"
    }
    def visualize_origin(origin):
        if isinstance(origin, TextPart): 
            return "t"
        elif isinstance(origin, PropertyKeysOrigin):
            p_keys = origin.property_keys
            if isinstance(p_keys, PropertyKey):
                return p_type_rep[origin.property_keys.property_type]
            elif isinstance(p_keys, tuple):
                return p_type_rep[p_keys[0].property_type]
    return "".join([visualize_origin(o) for o in origins])


In [6]:
generator = default_generator()
instantiator = default_instantiator(dataversion=dataversion)
ps_template_sampler, ps_answers_template_sampler, ps_renderer, rt_template_sampler, rt_renderer = default_templates_and_samplers(dataversion=dataversion)
vt_reorder_sampler = VariableTimeBasedSampler()
_seed = seed

In [7]:
_seed += 1
tree = generator.generate(seed=_seed)

sampler = CanonicalOrderSampler()
problem = sampler.sample_order(tree, seed=_seed)

instantiation = instantiator.instantiate(tree, seed=_seed) # NOTE: can instantiate on solved tree as the set of properties remain the same

nl, metadata = render_problem_and_answers(tree, instantiation, problem, ps_template_sampler, ps_answers_template_sampler, ps_renderer, seed=_seed)
print(f"Body: {problem.body_node_ids}\n")
print(nl.replace(". ", ".\n").replace("? ", "?\n\n"))

Body: [6, 7, 5, 3]

Henry has 10 chairs.
Henry has 2 chairs less than John has paintings.
Then, John looses 2 paintings to Grace.
John has 5 paintings more than Abigail.
How many paintings does Abigail have?

Abigail has 5 paintings.


In [8]:
_seed += 1
reordered_problem = vt_reorder_sampler.sample_order(tree, seed=_seed)
reo_nl, reo_meta = render_problem_and_answers(tree, instantiation, reordered_problem, ps_template_sampler, ps_answers_template_sampler, ps_renderer, seed=_seed)
reo_rt, reo_rt_meta = render_reasoning_trace(tree, instantiation, reordered_problem, rt_template_sampler, rt_renderer, seed=_seed)
print(f"Body: {reordered_problem.body_node_ids}\n")
print(reo_nl.replace(". ", ".\n").replace("? ", "?\n\n"))
print()
print(reo_rt)

Body: [7, 6, 5, 3]

Henry has 2 chairs less than John has paintings.
Henry has 10 chairs.
Then, Grace gets 2 paintings from John.
John has 5 paintings more than Abigail.
How many paintings does Abigail have?

Abigail has 5 paintings.

Henry has 10 chairs. Henry has 2 chairs less than John has paintings. Therefore, John now has 10 + 2 = 12 paintings.
Then, Grace gets 2 paintings from John. Therefore, John now has 12 - 2 = 10 paintings.
John has 5 paintings more than Abigail. Therefore, Abigail now has 10 - 5 = 5 paintings.
